# LOADING CSV

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/06-loading-structured-data.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/06-loading-structured-data.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/06-loading-structured-data.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table>

### Connect to EvaDB

In [1]:
%pip install evadb
import evadb
cursor = evadb.connect().cursor()

Note: you may need to restart the kernel to use updated packages.


### Create a Meta-data Table

In [2]:
create_table_query = f"""
    CREATE TABLE IF NOT EXISTS bddtestmeta(
        id INTEGER UNIQUE,
        frame_id INTEGER,
        video_id INTEGER,
        dataset_name TEXT(30),
        label TEXT(30),
        bbox NDARRAY FLOAT32(4),
        object_id INTEGER
    );
    
    """

cursor.query(create_table_query).df()

06-07-2023 18:14:02 WARNING[executor_utils:executor_utils.py:handle_if_not_exists:0084] Table: bddtestmeta already exists


""


### Download Berkeley Deepdrive Videos

In [3]:
# sample dataset of 2 videos
!wget -nc "https://www.dropbox.com/s/kg1q69ijbhjfecw/bddtest.zip"

# unzip
!unzip -n bddtest.zip

File ‘bddtest.zip’ already there; not retrieving.



Archive:  bddtest.zip


### Load 2 CSV files

In [4]:
cursor.load('./bddtest/info/bddtest_00a2e3ca5c856cde.csv', 'bddtestmeta', format="CSV").df()
cursor.load('./bddtest/info/bddtest_00a395fed60c0b47.csv', 'bddtestmeta', format="CSV").df()

,CSV,Number of loaded frames
0,bddtest/info/bddtest_00a395fed60c0b47.csv,4355


## Load the videos

In [5]:
cursor.query('DROP TABLE IF EXISTS bddtest_1;').df()
cursor.query('DROP TABLE IF EXISTS bddtest_2;').df()

cursor.load("./bddtest/videos/bddtest_00a2e3ca5c856cde.mp4", "bddtest_1", "video").df()
cursor.load("./bddtest/videos/bddtest_00a395fed60c0b47.mp4", "bddtest_2", "video").df()

,0
0,Number of loaded VIDEO: 1


## Run a Query over the Video dataset


In [6]:
query = cursor.table("bddtest_1")
query = query.filter("id < 3")
query = query.select("id, Yolo(data)")

query.df()

2023-06-07 18:14:09,405	INFO worker.py:1625 -- Started a local Ray instance.


,bddtest_1.id,yolo.labels,yolo.bboxes,yolo.scores
0,0,"[car, stop sign, car, car, car, car, car, car,...","[[490.404541015625, 332.927001953125, 716.9006...","[0.93, 0.91, 0.87, 0.82, 0.77, 0.73, 0.73, 0.5..."
1,1,"[car, car, stop sign, car, car, car, car, car,...","[[486.3106994628906, 331.76861572265625, 716.0...","[0.92, 0.87, 0.85, 0.76, 0.74, 0.73, 0.62, 0.5..."
2,2,"[car, stop sign, car, car, car, car, car, car,...","[[481.7199401855469, 331.24237060546875, 715.6...","[0.93, 0.9, 0.87, 0.79, 0.77, 0.74, 0.55, 0.54..."


## Run a query over the Meta-data Table

In [7]:
query = cursor.table("bddtestmeta")
query = query.filter("bddtestmeta.video_id = 4")
query = query.select("*")

query.df()

,bddtestmeta._row_id,bddtestmeta.id,bddtestmeta.frame_id,bddtestmeta.video_id,bddtestmeta.dataset_name,bddtestmeta.label,bddtestmeta.bbox,bddtestmeta.object_id
0,1,8185,0,4,bddtest,car,"[491.7783, 320.9238, 717.0901, 512.1478]",512
1,2,8186,0,4,bddtest,car,"[928.2679, 298.47574, 1279.5381, 571.17786]",513
2,3,8187,0,4,bddtest,car,"[816.0277, 347.52887, 1029.6998, 474.7344]",514
3,4,8188,0,4,bddtest,car,"[811.87067, 325.91223, 948.2217, 408.2217]",515
4,5,8189,0,4,bddtest,car,"[673.0254, 325.08084, 751.17786, 367.48267]",516
...,...,...,...,...,...,...,...,...
6797,11153,11581,1197,4,bddtest,car,"[554.96533, 335.88916, 588.2217, 378.291]",643
6798,11154,11582,1197,4,bddtest,car,"[693.8106, 350.0231, 734.5497, 395.75058]",644
6799,11155,11583,1197,4,bddtest,car,"[683.83374, 345.03464, 733.71826, 389.93073]",642
6800,11156,11584,1197,4,bddtest,car,"[676.351, 346.69745, 699.6305, 384.11084]",641
